<a href="https://colab.research.google.com/github/nxxk23/PROJECT/blob/main/VisitFile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
hemo = pd.read_csv('/content/drive/MyDrive/ckdresearch2023/Hemodialysis.csv', encoding='latin1')

In [4]:
visit = pd.read_csv('/content/drive/MyDrive/ckdresearch2023/Visit.csv')
visit.head()

<ipython-input-4-d66278be1d30>:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  visit = pd.read_csv('/content/drive/MyDrive/ckdresearch2023/Visit.csv')


,PatientUID,PatientvisitUID,Sex,Age,ENTY,NATNL,StartDTTM,EndDttm,Location,PricipalProblemCode,PricipalProblemName
0,19319,25325,หญิง,83,Out Patient,ไทย,2555-01-01 00:00:00.000,2557-11-26 23:58:59.730,NaN,NaN,NaN
1,83224,25327,ชาย,64,Out Patient,ไทย,2555-01-01 00:00:00.000,2557-11-26 23:58:59.730,NaN,NaN,NaN
2,160455,25336,ชาย,86,Out Patient,ไทย,2555-01-01 00:00:00.000,2557-11-26 23:58:59.730,NaN,NaN,NaN
3,181216,25338,หญิง,82,Out Patient,ไทย,2555-01-01 00:00:00.000,2557-11-26 23:58:59.730,NaN,NaN,NaN
4,568147,25357,ชาย,68,Out Patient,ไทย,2555-01-01 00:00:00.000,2557-11-26 23:58:59.730,NaN,NaN,NaN


In [5]:
# Create a list from hemo['PatientUID']
patient_uid_list = hemo['PatientUID'].unique().tolist()
# Remove patient UIDs from visit['PatientUID'] that are not in the list
visit = visit[~visit['PatientUID'].isin(patient_uid_list)]
visit.shape

(1195332, 11)

### **notfillna method**

In [6]:
import pandas as pd

selected = visit[['PatientUID', 'PatientvisitUID', 'Sex', 'Age', 'ENTY', 'NATNL', 'Location', 'PricipalProblemCode', 'PricipalProblemName']]

# Step 1: Group by 'PatientUID' and 'Sex' and count 'PatientvisitUID'
grouped = selected.groupby(['PatientUID', 'Sex']).agg(
    PatientvisitUID_count=('PatientvisitUID', 'count'),
    AgeMin=('Age', 'min'),
    AgeMax=('Age', 'max'),
    NATNL=('NATNL', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    Location_mode=('Location', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    PricipalProblemCode_mode=('PricipalProblemCode', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    PricipalProblemName_mode=('PricipalProblemName', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    InPatient_count=('ENTY', lambda x: (x == 'InPatient').sum()),
    OutPatient_count=('ENTY', lambda x: (x == 'Out Patient').sum()),
    EmergencyPatient_count=('ENTY', lambda x: (x == 'E').sum())
)

# Reset index to convert 'PatientUID' and 'Sex' from index to columns
grouped = grouped.reset_index()

In [7]:
grouped[grouped['PricipalProblemName_mode'].isnull()]

,PatientUID,Sex,PatientvisitUID_count,AgeMin,AgeMax,NATNL,Location_mode,PricipalProblemCode_mode,PricipalProblemName_mode,InPatient_count,OutPatient_count,EmergencyPatient_count
4145,291919,ชาย,7,59,65,ไทย,อายุรกรรม 8 (OPD)-D07,None,None,0,7,0
5320,483476,หญิง,9,49,57,ไทย,หน่วยทะเบียนกลาง(Test พิเศษ)เวชศาสตร์ชันสูตร,None,None,0,9,0
6205,591412,ชาย,3,62,64,ไทย,ทันตกรรม (คลินิก)-D1601,None,None,0,3,0
17102,2299587,ชาย,3,60,62,ไทย,หน่วยทะเบียนกลาง(Test พิเศษ)เวชศาสตร์ชันสูตร,None,None,0,3,0
17513,2430540,ชาย,2,54,55,ไทย,คลินิกตรวจสุขภาพเคลื่อนที่(อาชีวเวชศาสตร์) -D1705,None,None,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...
20501,3215454,หญิง,2,66,67,None,หน่วยทะเบียนกลาง(Test พิเศษ)เวชศาสตร์ชันสูตร,None,None,0,2,0
20522,3223365,หญิง,2,64,64,ไทย,หน่วยทะเบียนกลาง(Test พิเศษ)เวชศาสตร์ชันสูตร,None,None,0,2,0
20747,3313563,หญิง,2,89,89,ไทย,ทันตกรรม (คลินิก)-D1601,None,None,0,2,0
20773,3325468,หญิง,2,66,66,None,หน่วยทะเบียนกลาง(Test พิเศษ)เวชศาสตร์ชันสูตร,None,None,0,2,0


In [8]:
grouped

,PatientUID,Sex,PatientvisitUID_count,AgeMin,AgeMax,NATNL,Location_mode,PricipalProblemCode_mode,PricipalProblemName_mode,InPatient_count,OutPatient_count,EmergencyPatient_count
0,2299,หญิง,83,69,79,ไทย,เวชปฏิบัติทั่วไป (OPD)-D03,239873007,Osteoarthritis of knee,1,82,0
1,2304,หญิง,67,52,63,ไทย,ความดันโลหิตสูงวิกฤต (คลินิก)-DV193,1201005,Benign essential hypertension,1,63,3
2,2312,หญิง,83,54,65,ไทย,บูรณาการนอกเวลา (คลินิก)-OD191,431856006,Chronic kidney disease stage 2,1,82,0
3,2354,ชาย,64,60,71,ไทย,Pharmacy Health and Spa (Department)-D3101,U5732,ลมปลายปัตฆาตสัญญาณ 3 หลัง,0,62,2
4,2362,ชาย,88,54,65,ไทย,บูรณาการนอกเวลา (คลินิก)-OD191,E119,Non-insulin-dependent diabetes mellitus type 2...,1,86,1
...,...,...,...,...,...,...,...,...,...,...,...,...
20962,3421578,ชาย,6,66,67,ไทย,เวชปฏิบัติทั่วไป (OPD)-D03,I635,Cerebral infarction due to unspecified occlusi...,0,6,0
20963,3426284,หญิง,4,78,79,ลาว,บูรณาการนอกเวลา (คลินิก)-OD191,431857002,Chronic kidney disease stage 4,0,4,0
20964,3431501,หญิง,4,51,52,ไทย,ศัลยกรรมทั่วไป 1 (คลินิก)-D601,K805,Calculus of bile duct without cholangitis or c...,0,4,0
20965,3432056,หญิง,14,73,74,ไทย,เวชศาสตร์นิวเคลียร์ (OPD)-D15,127018007,Malignant neoplasm of thyroid gland,4,10,0


#### **Check ENTY**
```
ถ้าเราไม่ใช้ Fillna Method เราจะเจอคอลัมน์ ENTY ทีเ่ป็นค่าว่าง และไม่ถูกเติมด้วย mode
มีอยุ่ 1 row ก็จริง แต่นั่นหมายความถึงวิธีการใช้ ถ้าสมมติมันมีค่ามากกว่านี้
เราก็จะไม่มีทางรู้เลยเพราะเราไม่ได้หยิบออกมาเช็คทีละ Patient
```



In [9]:
grouped['TotalCountCheck'] = grouped['InPatient_count'] + grouped['OutPatient_count'] + grouped['EmergencyPatient_count'] == grouped['PatientvisitUID_count']
grouped[grouped['TotalCountCheck'] == False]

,PatientUID,Sex,PatientvisitUID_count,AgeMin,AgeMax,NATNL,Location_mode,PricipalProblemCode_mode,PricipalProblemName_mode,InPatient_count,OutPatient_count,EmergencyPatient_count,TotalCountCheck
13698,1392530,หญิง,144,29,40,ไทย,โรคไตทั่วไป (คลินิก)-D723,236436003,End stage renal failure with renal transplant,9,131,3,False


In [10]:
visit.loc[(visit['PatientUID'] == 1392530) & (visit['ENTY'].isnull())]

,PatientUID,PatientvisitUID,Sex,Age,ENTY,NATNL,StartDTTM,EndDttm,Location,PricipalProblemCode,PricipalProblemName
1222241,1392530,9607138,หญิง,39,NaN,ไทย,2565-06-29 16:24:28.713,2565-06-30 05:01:01.980,NaN,NaN,NaN


#### **Check NATNL**

In [11]:
grouped[grouped['NATNL'].isnull()]

,PatientUID,Sex,PatientvisitUID_count,AgeMin,AgeMax,NATNL,Location_mode,PricipalProblemCode_mode,PricipalProblemName_mode,InPatient_count,OutPatient_count,EmergencyPatient_count,TotalCountCheck
164,11276,ชาย,49,76,81,None,โรคระบบประสาทและสมอง (คลินิก)-D713,44054006,Secondary parkinsonism,0,49,0,True
602,30758,ชาย,3,31,40,None,OPD AE-DV191,161587009,H/O: head injury,1,1,1,True
1409,91243,หญิง,21,81,83,None,อายุรกรรม 8 (OPD)-D07,I10,Essential (primary) hypertension,4,17,0,True
1610,107792,หญิง,26,63,68,None,โรคระบบประสาทและสมอง (คลินิก)-D713,44054006,Type 2 diabetes mellitus,0,26,0,True
2632,156981,ชาย,13,79,86,None,ศัลยศาสตร์อุบัติเหตุ - D611,196828002,Acute cholangitis,2,11,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20354,3174908,หญิง,3,74,76,None,หน่วยทะเบียนกลาง(Test พิเศษ)เวชศาสตร์ชันสูตร,None,None,0,3,0,True
20499,3215446,หญิง,2,74,74,None,หน่วยทะเบียนกลาง(Test พิเศษ)เวชศาสตร์ชันสูตร,None,None,0,2,0,True
20500,3215452,หญิง,2,74,75,None,หน่วยทะเบียนกลาง(Test พิเศษ)เวชศาสตร์ชันสูตร,None,None,0,2,0,True
20501,3215454,หญิง,2,66,67,None,หน่วยทะเบียนกลาง(Test พิเศษ)เวชศาสตร์ชันสูตร,None,None,0,2,0,True


#### **Check Location**

In [12]:
grouped[grouped['Location_mode'].isnull()]

,PatientUID,Sex,PatientvisitUID_count,AgeMin,AgeMax,NATNL,Location_mode,PricipalProblemCode_mode,PricipalProblemName_mode,InPatient_count,OutPatient_count,EmergencyPatient_count,TotalCountCheck


#### **Check PricipalProblemCode & Name**

In [13]:
grouped[grouped['PricipalProblemCode_mode'].isnull()]

,PatientUID,Sex,PatientvisitUID_count,AgeMin,AgeMax,NATNL,Location_mode,PricipalProblemCode_mode,PricipalProblemName_mode,InPatient_count,OutPatient_count,EmergencyPatient_count,TotalCountCheck
4145,291919,ชาย,7,59,65,ไทย,อายุรกรรม 8 (OPD)-D07,None,None,0,7,0,True
5320,483476,หญิง,9,49,57,ไทย,หน่วยทะเบียนกลาง(Test พิเศษ)เวชศาสตร์ชันสูตร,None,None,0,9,0,True
6205,591412,ชาย,3,62,64,ไทย,ทันตกรรม (คลินิก)-D1601,None,None,0,3,0,True
17102,2299587,ชาย,3,60,62,ไทย,หน่วยทะเบียนกลาง(Test พิเศษ)เวชศาสตร์ชันสูตร,None,None,0,3,0,True
17513,2430540,ชาย,2,54,55,ไทย,คลินิกตรวจสุขภาพเคลื่อนที่(อาชีวเวชศาสตร์) -D1705,None,None,0,2,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20501,3215454,หญิง,2,66,67,None,หน่วยทะเบียนกลาง(Test พิเศษ)เวชศาสตร์ชันสูตร,None,None,0,2,0,True
20522,3223365,หญิง,2,64,64,ไทย,หน่วยทะเบียนกลาง(Test พิเศษ)เวชศาสตร์ชันสูตร,None,None,0,2,0,True
20747,3313563,หญิง,2,89,89,ไทย,ทันตกรรม (คลินิก)-D1601,None,None,0,2,0,True
20773,3325468,หญิง,2,66,66,None,หน่วยทะเบียนกลาง(Test พิเศษ)เวชศาสตร์ชันสูตร,None,None,0,2,0,True


In [14]:
grouped[grouped['PricipalProblemName_mode'].isnull()]

,PatientUID,Sex,PatientvisitUID_count,AgeMin,AgeMax,NATNL,Location_mode,PricipalProblemCode_mode,PricipalProblemName_mode,InPatient_count,OutPatient_count,EmergencyPatient_count,TotalCountCheck
4145,291919,ชาย,7,59,65,ไทย,อายุรกรรม 8 (OPD)-D07,None,None,0,7,0,True
5320,483476,หญิง,9,49,57,ไทย,หน่วยทะเบียนกลาง(Test พิเศษ)เวชศาสตร์ชันสูตร,None,None,0,9,0,True
6205,591412,ชาย,3,62,64,ไทย,ทันตกรรม (คลินิก)-D1601,None,None,0,3,0,True
17102,2299587,ชาย,3,60,62,ไทย,หน่วยทะเบียนกลาง(Test พิเศษ)เวชศาสตร์ชันสูตร,None,None,0,3,0,True
17513,2430540,ชาย,2,54,55,ไทย,คลินิกตรวจสุขภาพเคลื่อนที่(อาชีวเวชศาสตร์) -D1705,None,None,0,2,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20501,3215454,หญิง,2,66,67,None,หน่วยทะเบียนกลาง(Test พิเศษ)เวชศาสตร์ชันสูตร,None,None,0,2,0,True
20522,3223365,หญิง,2,64,64,ไทย,หน่วยทะเบียนกลาง(Test พิเศษ)เวชศาสตร์ชันสูตร,None,None,0,2,0,True
20747,3313563,หญิง,2,89,89,ไทย,ทันตกรรม (คลินิก)-D1601,None,None,0,2,0,True
20773,3325468,หญิง,2,66,66,None,หน่วยทะเบียนกลาง(Test พิเศษ)เวชศาสตร์ชันสูตร,None,None,0,2,0,True


## **Store Excel File**

In [15]:
grouped = grouped[['PatientUID','Sex','NATNL','AgeMin','AgeMax','PatientvisitUID_count','InPatient_count','OutPatient_count','EmergencyPatient_count','Location_mode','PricipalProblemCode_mode','PricipalProblemName_mode']]
grouped

,PatientUID,Sex,NATNL,AgeMin,AgeMax,PatientvisitUID_count,InPatient_count,OutPatient_count,EmergencyPatient_count,Location_mode,PricipalProblemCode_mode,PricipalProblemName_mode
0,2299,หญิง,ไทย,69,79,83,1,82,0,เวชปฏิบัติทั่วไป (OPD)-D03,239873007,Osteoarthritis of knee
1,2304,หญิง,ไทย,52,63,67,1,63,3,ความดันโลหิตสูงวิกฤต (คลินิก)-DV193,1201005,Benign essential hypertension
2,2312,หญิง,ไทย,54,65,83,1,82,0,บูรณาการนอกเวลา (คลินิก)-OD191,431856006,Chronic kidney disease stage 2
3,2354,ชาย,ไทย,60,71,64,0,62,2,Pharmacy Health and Spa (Department)-D3101,U5732,ลมปลายปัตฆาตสัญญาณ 3 หลัง
4,2362,ชาย,ไทย,54,65,88,1,86,1,บูรณาการนอกเวลา (คลินิก)-OD191,E119,Non-insulin-dependent diabetes mellitus type 2...
...,...,...,...,...,...,...,...,...,...,...,...,...
20962,3421578,ชาย,ไทย,66,67,6,0,6,0,เวชปฏิบัติทั่วไป (OPD)-D03,I635,Cerebral infarction due to unspecified occlusi...
20963,3426284,หญิง,ลาว,78,79,4,0,4,0,บูรณาการนอกเวลา (คลินิก)-OD191,431857002,Chronic kidney disease stage 4
20964,3431501,หญิง,ไทย,51,52,4,0,4,0,ศัลยกรรมทั่วไป 1 (คลินิก)-D601,K805,Calculus of bile duct without cholangitis or c...
20965,3432056,หญิง,ไทย,73,74,14,4,10,0,เวชศาสตร์นิวเคลียร์ (OPD)-D15,127018007,Malignant neoplasm of thyroid gland


In [16]:
grouped.to_excel('VisitCount.xlsx')